In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
from scipy.optimize import linprog
from tqdm import tqdm

import os
import torch
from scipy.linalg import LinAlgWarning, LinAlgError
from scipy.optimize._optimize import OptimizeWarning
from scipy.optimize._linprog_util import _clean_inputs, _get_Abc
import warnings
import numpy as np

from generate_instances_lp import generate_setcover, Graph, generate_indset, generate_cauctions, generate_capacited_facility_location

In [3]:
rng = np.random.RandomState(1)

In [4]:
from scipy.linalg import qr
from torch_geometric.data import Batch, HeteroData, InMemoryDataset
from collections import namedtuple

_LPProblem = namedtuple('_LPProblem',
                        'c A_ub b_ub A_eq b_eq bounds x0 integrality')
_LPProblem.__new__.__defaults__ = (None,) * 7  # make c the only required arg

In [5]:
def normalize_cons(A, b):
    if A is None or b is None:
        return A, b
    Ab = np.concatenate([A, b[:, None]], axis=1)
    max_logit = np.abs(Ab).max(axis=1)
    max_logit[max_logit == 0] = 1.
    Ab = Ab / max_logit[:, None]
    A = Ab[:, :-1]
    b = Ab[:, -1]
    return A, b

In [6]:
def surrogate_gen(m, n, d):
    A = np.random.randn(m, n)
    A[np.random.rand(m, n) > d] = 0.
    x_feas = np.abs(np.random.randn(n))  # Ensure x_feas is non-negative
    b = A @ x_feas + np.abs(np.random.randn(m)) * 10  # Ensure feasibility

    c = np.abs(np.random.randn(n))
    return A, b, c

In [7]:
A, b, c = surrogate_gen(50, 100, 0.1)
c = c / (np.abs(c).max() + 1.e-10)  # does not change the result
A, b = normalize_cons(A, b)
bounds = None

In [8]:
m, n = A.shape

In [9]:
# lp = _LPProblem(c, A, b, None, None, None, None, None)
# lp = _clean_inputs(lp)
# A, b, c, *_ = _get_Abc(lp, 0.)

In [10]:
A.shape

(50, 100)

In [11]:
np.linalg.matrix_rank(A)

50

In [12]:
res = linprog(c, A_ub=A, b_ub=b, bounds=None, method='highs')
res.x.max()

3.518963395737572

In [13]:
def active_idx(A, x, b, eps=1.e-8):
    vio = A @ res.x - b
    assert vio.max() <= eps
    vio_mask = np.abs(vio) < eps
    return np.where(vio_mask)[0]

# A @ c + b smaller, more likely to be active

In [ ]:
actives = active_idx(A, res.x, b)

In [ ]:
actives

In [ ]:
actives.shape[0], m

In [ ]:
# heur = A @ c + b
# heur

In [ ]:
heur

In [ ]:
topk = 10

In [ ]:
sortd_Ac_add_b = np.argsort(heur)

In [ ]:
preds = np.isin(sortd_Ac_add_b[:topk], actives)

In [ ]:
preds.sum() / preds.shape[0]

In [ ]:
preds = np.isin(sortd_Ac_add_b[-topk:], actives)

In [ ]:
preds.sum() / preds.shape[0]

In [ ]:
actives

In [ ]:
np.isin(sortd_Ac_add_b, actives).astype(np.int32)

# todo: sensitive analys, for example, add some bias to cons 6, does it change x solution? (6 is not active, but also may affect)

feasible point

# TODO:

split this into l2, l1, log

In [ ]:
def find_feas(A, b, c, x0, maxiter = 1000):
    m = np.zeros_like(x0)
    v = np.zeros_like(x0)
    beta1 = 0.9
    beta2 = 0.999
    learning_rate = 0.1
    eps = 1.e-8

    pbar = tqdm(range(maxiter))
    for i in pbar:
        residual = A @ x0 - b
        bias = residual.max()
    
        pbar.set_postfix({'res': residual.max(), 'x': x0.min()})
        if residual.max() <= -1. and x0.min() >= 0.:
            break
        
        # grad = A.T @ np.where(residual > 0, residual, 0.) - np.where(x0 < 0, x0, 0.)
        # grad = A.T @ np.where(residual > 0, 1, 0.) - np.where(x0 < 0, -1, 0.)
        grad = A.T @ (1. / (-residual + bias + 0.1)) - np.where(x0 < 0, x0, 0.)
        m = (1 - beta1) * grad + beta1 * m  # first  moment estimate.
        v = (1 - beta2) * (grad ** 2) + beta2 * v  # second moment estimate.
        mhat = m / (1 - beta1**(i + 1))  # bias correction.
        vhat = v / (1 - beta2**(i + 1))
        x0 = x0 - learning_rate * mhat / (np.sqrt(vhat) + eps)
        x0[x0 < 0] = 0.
    return x0

In [ ]:
x = find_feas(A, b, c, np.abs(np.random.randn(c.shape[0])), 1000)

In [ ]:
np.argmax(A @ (x - a * c) - b)

In [ ]:
np.argmax(A @ x - b)

In [ ]:
A[46]

In [ ]:
np.abs()

In [ ]:
def find_close_cons(x, A, b, c):
    neg_mask = (-c < 0)
    steps1 = x[neg_mask] / c[neg_mask]
    if len(steps1):
        alpha1 = steps1.min()
    else:
        alpha1 = 1.e8
    
    neg_mask = (A @ c < 0)
    steps2 = (A @ x - b)[neg_mask] / (A @ c - 1.e-10)[neg_mask]
    if len(steps2):
        alpha2 = steps2.min()
    else:
        alpha2 = 1.e8

    a = min(alpha1, alpha2)
    assert (A @ (x - a * c) - b).max() <= 0.
    return a

In [ ]:
a = find_close_cons(x, A, b, c)

In [ ]:
np.argmax(A @ (x - a * c) - b)

In [ ]:
np.argmax(A @ x - b)

In [ ]:
a = find_close_cons(x, A, b, np.eye(x.shape[0])[4])

In [ ]:
np.argmax(A @ (x - a * c) - b)

In [ ]:
np.argmax(A @ x - b)

observed problem: if a point is too close to a cons, then its closest cons is pretty much the same as its steepest cons!  

solution 1: find an orthogonal direction (heuristic: find Ai @ eye, argmin(abs))  
solution 2: push it further into the center of the feasible region, use sum log loss func for grad descent